<a href="https://colab.research.google.com/github/Alexis3004/estadistica/blob/master/Estad%C3%ADstica_No_Param%C3%A9trica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Métodos No Paramétricos en Estadística

<p align="justify">Con base en los datos trabajados en el módulo de muestreo (archivo adjunto en el enlace) extraiga una muestra representativa (calcule el tamaño para un nivel de confianza de 89% y un error de 3% y use ese tamaño específico) y determine si la clasificación en niveles de desempeño en dos de las cinco competencias genéricas son producto del azar o no.

Determine si esos resultados son independientes o dependientes entre sí.
</p>
<p align="justify">
Con la muestra, calcule un promedio general por estudiante para las cinco competencia y realice un ranking inicial y un ranking final por departamento; determine si hubo cambios significativos en esa clasificación</p>

[Descargue el archivo aquí](https://www.dropbox.com/s/xgsf3l5gjw0h0q4/saberpro.RData?dl=1)


### Tratamiento de datos

In [ ]:
library(devtools)

In [ ]:
load(url('https://www.dropbox.com/s/mp094kybobr2ubs/saberpro.RData?dl=1'))
ls()

In [ ]:
install.packages('coin')
install.packages('plyr')
install.packages('samplingbook')
install_github('DFJL/samplingUtil')
install.packages("magrittr")
install.packages("dplyr")

In [ ]:
library(plyr)
library(samplingbook) # llamamos la librería
library(SamplingUtil)
library(magrittr) # Permite leer la funcion %>% que se usa para asignar funciones a objetos
library(dplyr) # Contiene la funcion select
library(coin)
library(tidyverse)

In [ ]:
sp161718 <- sp161718[is.na(sp161718$raz_cuant_punt) == FALSE & is.na(sp161718$lect_crit_punt) == FALSE & is.na(sp161718$comp_ciud_punt) == FALSE & is.na(sp161718$ingles_punt) == FALSE & is.na(sp161718$comu_escr_punt) == FALSE,]

In [ ]:

saberPro <- data.frame(
  index = 1:nrow(sp161718),
  periodo = sp161718$periodo,
  departamento = sp161718$estu_depto_reside,
  RC = sp161718$raz_cuant_punt,
  PRC = sp161718$raz_cuant_des,
  LC = sp161718$lect_crit_punt,
  PLC = sp161718$lect_crit_des,
  CC = sp161718$comp_ciud_punt,
  PCC = sp161718$comp_ciud_des,
  IN = sp161718$ingles_punt,
  PIN = sp161718$ingles_desem,
  CE = sp161718$comu_escr_punt,
  PCE = sp161718$comu_escr_des
)

In [ ]:
primer <- substring(saberPro$periodo, 1,4) #A la serie periodo de dejamos sólo el año
saberPro$periodo <- as.character(saberPro$periodo)  #convertimos la serie periodo a caracteres
saberPro$periodo <- primer
saberPro[,3]<-revalue(saberPro[,'departamento'],c('NARIÃƒâ€˜O'="NARIÑO",'NARIÃ‘O'="NARIÑO"))
saberPro$periodo <- as.factor(saberPro$periodo)
saberPro$PRC = as.factor(saberPro$PRC) 
saberPro$PLC = as.factor(saberPro$PLC) 
saberPro$PCC = as.factor(saberPro$PCC) 
saberPro$PIN = as.factor(saberPro$PIN) 
saberPro$PCE = as.factor(saberPro$PCE) 

In [ ]:
head(saberPro)

,index,periodo,departamento,RC,PRC,LC,PLC,CC,PCC,IN,PIN,CE,PCE
,<int>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>
1,1,2016,ATLANTICO,160,3,130,2,167,3,160,A2,155,3
2,2,2016,CUNDINAMARCA,138,2,132,2,77,1,125,A1,164,3
3,3,2016,CALDAS,146,2,157,2,150,2,104,A1-,122,2
4,4,2016,MAGDALENA,160,3,172,3,151,2,160,A2,192,4
5,5,2016,CORDOBA,130,2,122,1,144,2,192,B1,131,2
6,6,2016,CAUCA,159,3,129,2,147,2,108,A1-,154,3


Realizamos un muestro con un nivel de confianza del 89% y un error del 3%

In [ ]:
#error del 3%, probabilidad de seleccionar los datos del 50%, nivel de confianza del 89%
muestra <- sample.size.prop(e=0.03, P = 0.5, N = length(saberPro$RC), level = 0.99)$n
P1<- sample(1:nrow(saberPro),size=muestra,replace=FALSE)

Realizamos los estratos por departamento, pero también teniendo en cuenta el periodo

In [ ]:
Estratos<- saberPro %>%
  select(departamento,periodo) %>%
  group_by(departamento,periodo) %>%
  summarise(n=n()) %>%
  mutate(p=(n/725295))
#Hallamos el tamaño de la muestra para cada estrato dependiendo de la muestra total
P2<-nstrata(n=muestra,wh=Estratos[,4],method="proportional")
Estra<-data.frame(fac1=Estratos$departamento,fac2=Estratos$periodo,n=Estratos$n,P2)

`summarise()` regrouping output by 'departamento' (override with `.groups` argument)



P3 es nuestra muestra por deartamentos y por periodos de tiempo

In [ ]:
P3 <- c()
a = c(4:nrow(Estra))
for (i in a){
  #construimos un vector con los índices que pertenecen a un departamento en específico y a un periodo en específico
  vec <- saberPro[saberPro$periodo==Estra[i,2] & saberPro$departamento==Estra[i,1],'index']
  sampl <- sample(vec,size=Estra[i,4],replace=FALSE)
  #Construimos un vector que nos permita seleccionar la muestra:
  P3<-append(P3,sampl)
}

### Prueba de Bondad de Ajuste

determine si la clasificación en niveles de desempeño en dos de las cinco competencias genéricas son producto del azar o no.

Las hipótesis

* $H_{0}:$ No hay diferencia entre las proporciones de las categorías.
* $H_{1}:$ Existe diferencia entre las proporciones de las categorías.

#####Observemos si los datos del Nivel de Desempeño Razonamiento Cuantitativo (PRC) son producto del azar.

In [ ]:
table(saberPro[P3,c('PRC')])


  1   2   3   4 
406 609 748  76 

In [ ]:
chisq.test(table(saberPro[P3,c('PRC')]))


	Chi-squared test for given probabilities

data:  table(saberPro[P3, c("PRC")])
X-squared = 555.77, df = 3, p-value < 2.2e-16


In [ ]:
with(chisq.test(table(saberPro[P3,c('PRC')])), data.frame(observed,expected))

,Var1,Freq,expected
,<fct>,<int>,<dbl>
1,1,406,459.75
2,2,609,459.75
3,3,748,459.75
4,4,76,459.75


<p align="justify">Según la prueba X-squared, podemos observar que tenemos 3 grados de libertad y por el valor del P_Value rechazamos la hipótesis nula, por lo tanto concluimos que existe diferencia entre las proporciones de las categorías del nivel de desempeño de Razonamiento cuantitativo, además, debido a esto, es válido confirmar que <strong>los datos no son productos del Azar</strong>, por otro lado, vemos que el nivel de desempeño que más frecuencia posee, es el nivel 3 y que el nivel mas alto (4) es el que menor frecuencia presenta de todos los niveles</p>

#####Observemos si los datos del Nivel de Desempeño Lectura Crítica (PLC) son producto del azar.

In [ ]:
chisq.test(table(saberPro[P3,c('PLC')]))


	Chi-squared test for given probabilities

data:  table(saberPro[P3, c("PLC")])
X-squared = 491.17, df = 3, p-value < 2.2e-16


In [ ]:
with(chisq.test(table(saberPro[P3,c('PLC')])), data.frame(observed,expected))

,Var1,Freq,expected
,<fct>,<int>,<dbl>
1,1,365,459.75
2,2,706,459.75
3,3,653,459.75
4,4,115,459.75


<p align="justify">Según la prueba X-squared, podemos observar que tenemos 3 grados de libertad y por el valor del P_Value rechazamos la hipótesis nula, por lo tanto concluimos que existe diferencia entre las proporciones de las categorías del nivel de desempeño de Lectura Crítica, además, debido a esto, es válido confirmar que <strong>los datos no son productos del Azar</strong>, por otro lado, vemos que el nivel de desempeño que más frecuencia posee, es el nivel 2 y que el nivel mas alto (4) es el que menor frecuencia presenta de todos los niveles</p>

### Prueba de Independencia (Tablas de Contingencia)

Determine si esos resultados son independientes o dependientes entre sí.

Las hipótesis

* $H_{0}:$ PRC y PLC Son Independientes
* $H_{1}:$ PRC y PLC Son Dependientes

In [ ]:
PRC = table(saberPro[P3,c('PRC')])
PLC = table(saberPro[P3,c('PLC')])
A1 = as.data.frame(rbind(PRC, PLC))
A1

,1,2,3,4
,<int>,<int>,<int>,<int>
PRC,406,609,748,76
PLC,365,706,653,115


In [ ]:
with(saberPro[P3,], table(PRC,PLC))

   PLC
PRC   1   2   3   4
  1 175 168  59   4
  2 140 292 166  11
  3  47 243 383  75
  4   3   3  45  25

In [ ]:
with(saberPro[P3,], chisq.test(table(PLC,PRC)))

Warning message in chisq.test(table(PLC, PRC)):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  table(PLC, PRC)
X-squared = 504.86, df = 9, p-value < 2.2e-16


<p align="justify">Según el P_value del Pearson's Chi-squared test rechazamos la hipótesis nula y concluimos que el Nivel de Desempeño Lectura Crítica (PLC) y el Nivel de Desempeño Razonamiento Cuantitativo (PRC) son dependientes entre sí</p>

### Prueba de Signos (Datos Ordenados)

<p align="justify">Con la muestra, calcule un promedio general por estudiante para las cinco competencias y realice un ranking inicial y un ranking final por departamento; determine si hubo cambios significativos en esa clasificación</p>

Creamos un el dataset Puntaje con el periodo de la prueba, el departamento y el puntaje total obtenido por el departamento para las 5 pruebas en cada periodo

In [ ]:
Puntaje <- data.frame(
  periodo = saberPro[P3,'periodo'],
  departamento = saberPro[P3,'departamento'],
  puntajeTotal = ((saberPro[P3,'RC']+saberPro[P3,'LC']+saberPro[P3,'CC']+saberPro[P3,'IN']+saberPro[P3,'CE'])/5)
)

Agrupamos los datos por departamento y periodo de tal modo que sólo tengamos un dato puntaje para un departamento en un periodo determinado

In [ ]:
E<- Puntaje %>%
  select(departamento,periodo,puntajeTotal) %>%
  group_by(departamento,periodo) %>%
  summarise(puntajeGlobal=mean(puntajeTotal))

`summarise()` regrouping output by 'departamento' (override with `.groups` argument)



Construimos el dataset del puntaje obtenido en un periodo  con respecto al departamento

In [ ]:
p2017<-E[E$periodo=='2017',c('puntajeGlobal')]
p2018<-E[E$periodo=='2018',c('puntajeGlobal')]
p2016<-E[E$periodo=='2016',c('puntajeGlobal')]
total <- data.frame(departamento=unique(E$departamento),P2016 = p2016,P2017 = p2017,P2018=p2018)
names(total)<-c('departamento','P2016','P2017','P2018')

In [ ]:
O2016<-total[,c(1,2)] %>% arrange(desc(P2016)) %>% mutate(puesto2016 = row_number())
O2017<-total[,c(1,3)] %>% arrange(desc(P2017))  %>% mutate(puesto2017 = row_number())
O2018<-total[,c(1,4)] %>% arrange(desc(P2018))  %>% mutate(puesto2018 = row_number())
totalPuesto<- merge (O2016, O2017, by.x = "departamento",by.y = "departamento")
totalPuesto<- merge (totalPuesto, O2018, by.x = "departamento",by.y = "departamento")
totalPuesto <- totalPuesto %>% arrange(desc(P2016))
head(totalPuesto)

,departamento,P2016,puesto2016,P2017,puesto2017,P2018,puesto2018
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,SAN ANDRES,171.6000,1,125.2000,30,126.6000,31
2,ANTIOQUIA,158.3222,2,154.2389,5,149.2592,12
3,BOGOTA,158.0178,3,156.3874,3,157.2095,3
4,TOLIMA,153.1750,4,143.3333,17,141.1500,19
5,CUNDINAMARCA,153.1188,5,152.1576,7,148.0765,13
6,SANTANDER,153.0645,6,147.5600,10,155.2600,5


hipótesis:

* $H_{0}: mediana = 0$
* $H_{1}: mediana \neq 0$

Para los periodos 2016 y 2017

In [ ]:
wilcoxsign_test(puesto2016 ~ puesto2017, data = totalPuesto, distribution = "exact")


	Exact Wilcoxon-Pratt Signed-Rank Test

data:  y by x (pos, neg) 
	 stratified by block
Z = 0.40249, p-value = 0.6941
alternative hypothesis: true mu is not equal to 0


Para los periodos 2016 y 2018

In [ ]:
wilcoxsign_test(puesto2016 ~ puesto2018, data = totalPuesto, distribution = "exact")


	Exact Wilcoxon-Pratt Signed-Rank Test

data:  y by x (pos, neg) 
	 stratified by block
Z = 0.35087, p-value = 0.7322
alternative hypothesis: true mu is not equal to 0


Para los periodos 2017 y 2018

In [ ]:
wilcoxsign_test(puesto2017 ~ puesto2018, data = totalPuesto, distribution = "exact")


	Exact Wilcoxon-Pratt Signed-Rank Test

data:  y by x (pos, neg) 
	 stratified by block
Z = -0.10269, p-value = 0.9226
alternative hypothesis: true mu is not equal to 0


<p align="justify">Según el P_value de nuestos test exactos de Wilcoxon-Pratt no negamos la hipótesis nula y concluimos que la mediana es igual a cero o lo que significa que que no hubo cambios en el promedio general de la prueba para cada departamento en cada periodo, o que más de la mitad de los datos no cambiaron significativamente, por lo tanto podemos decir que los cambios que se presentaron, son probablemente producto del azar</p>

### Conclusiones

<p align="justify">Podemos concluir que los resultados de los diferentes niveles o categorías de desempeño de la prueba saber pro, son diferentes o existen diferencias entre ellas, por lo tanto afirmamos que estos resultados no son producto del azar, además que dichos resultados dependen unos de los otros y por lo tanto se pudo hallar que no han habito cambios estadísticamente significativos hablando del puntaje total que los departamentos han obtenido a través de los periodos 2016, 2017 y 2018 </p>
